## Game of life using dask

In [34]:
import dask
import numpy as np
from dask.distributed import Client
import dask.array as da
import dask.multiprocessing


In [36]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
# dask.config.set(scheduler='threads')
# client = Client()
client = Client(n_workers=6, threads_per_worker=6, processes=True, memory_limit='2.5GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51386/status,
Dashboard: http://127.0.0.1:51386/status,Workers: 6
Total threads: 36,Total memory: 13.97 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51387,Workers: 6
Dashboard: http://127.0.0.1:51386/status,Total threads: 36
Started: Just now,Total memory: 13.97 GiB
Comm: tcp://127.0.0.1:51429,Total threads: 6
Dashboard: http://127.0.0.1:51432/status,Memory: 2.33 GiB
Nanny: tcp://127.0.0.1:51390,


In [42]:

def read_input_file(input_file, chunk_size):
    # initialize board
    with open(input_file) as f:
        w, h = [int(x) for x in next(f).split()]
        # use the smallest data type
        board = np.zeros((w, h),dtype=np.uint8)
        for line_count, line in enumerate(f, start=0):
            single_numbers = line.split()
            x, y = map(int, single_numbers[:2])
            board[x][y] = 1
        dask_board = da.from_array(board, chunks=(chunk_size, chunk_size))
    return dask_board

In [43]:
def neighbors_number(board, row, col):
    # add plus two to create a proper iteration
    neighbors = board[max(0, row-1):min(board.shape[0], row+2), max(0, col-1):min(board.shape[1], col+2)]
    return np.sum(neighbors) - board[row, col]

def tick(board):
    # the copy takes a lot of space
    w, h = board.shape
    new_board = np.zeros((w, h),dtype=np.uint8)
    for r in range(w):
        for c in range(h):
            #for dead cell
            if board[r][c] == 0 and neighbors_number(board, r, c) == 3:
                new_board[r][c] = 1
            #for alive cell
            if board[r][c] == 1 and (neighbors_number(board, r, c) < 2 or neighbors_number(board, r, c) > 3):
                new_board[r][c] = 0
    return new_board


    

def play_game(dask_board, iterations):
    for i in range(iterations):
        dask_board = dask_board.map_overlap(tick, depth=1, boundary='none')
    final_board = dask_board.compute()
    return final_board


In [44]:
board = read_input_file('input_data/input_big.txt', 500)

In [45]:
%%time
warnings.filterwarnings('ignore')
print(play_game(board, 1))


[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
CPU times: total: 5min 1s
Wall time: 9min 11s


In [47]:
client.shutdown()